In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
df = pd.read_csv("../../labelled_data/part_D/imputed_partd.csv")

In [3]:
df = df.drop(["Unnamed: 0","PRSCRBR_NPI"],axis=1)

In [4]:
df

,Prscrbr_Crdntls,Prscrbr_Gndr,Prscrbr_Type,Prscrbr_Type_src,Tot_Clms,Tot_30day_Fills,Tot_Drug_Cst,Tot_Day_Suply,Tot_Benes,GE65_Sprsn_Flag,...,Bene_Race_Black_Cnt,Bene_Race_Api_Cnt,Bene_Race_Hspnc_Cnt,Bene_Race_Natind_Cnt,Bene_Race_Othr_Cnt,Bene_Dual_Cnt,Bene_Ndual_Cnt,Bene_Avg_Risk_Scre,Fraud,FraudType
0,7632,0.0,86,1.0,324.0,369.800000,20606.08,8621.0,106.0,1.0,...,27.0,0.0,13.0,0.0,0.0,28.0,78.0,2.245800,0,0
1,7632,0.0,9,1.0,1992.0,2145.666667,79803.65,60953.0,228.0,1.0,...,81.0,0.0,12.0,0.0,0.0,125.0,103.0,1.695165,0,0
2,4394,0.0,42,1.0,57.0,57.700000,327.34,554.0,43.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,43.0,1.006070,0,0
3,3535,1.0,42,1.0,18.0,18.000000,128.52,181.0,16.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,12.0,1.251869,0,0
4,6421,1.0,120,1.0,37.0,47.000000,3834.98,1366.0,17.0,1.0,...,0.0,0.0,0.0,0.0,0.0,11.0,11.0,4.919074,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254982,13080,1.0,128,1.0,40.0,80.666667,9870.01,2357.0,17.0,1.0,...,0.0,16.0,0.0,0.0,0.0,0.0,11.0,1.451353,0,0
1254983,4831,0.0,42,1.0,72.0,73.200000,397.84,1092.0,27.0,1.0,...,18.0,18.0,0.0,0.0,0.0,18.0,18.0,1.344691,0,0
1254984,15356,0.0,120,1.0,47.0,90.466667,13618.96,2704.0,21.0,1.0,...,0.0,0.0,0.0,0.0,0.0,13.0,13.0,1.545349,1,2
1254985,7632,0.0,137,1.0,249.0,291.066667,10899.52,5801.0,126.0,1.0,...,14.0,16.0,18.0,0.0,14.0,18.0,108.0,1.109445,0,0


In [5]:
X = df.drop(['Fraud', 'FraudType'], axis=1)
y = df['Fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Convert the data to DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set hyperparameters for the XGBoost model
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 3,
    'eta': 0.1,
    'subsample': 0.8,
}

# Train the XGBoost model
num_round = 100
model = xgb.train(params, dtrain, num_round)

# Make predictions on the test set
y_pred = model.predict(dtest)

# Convert predicted probabilities to binary predictions
y_pred = [1 if pred > 0.5 else 0 for pred in y_pred]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", classification_rep)

Accuracy: 0.9997649383660427

Confusion Matrix:
 [[250939      0]
 [    59      0]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    250939
           1       0.00      0.00      0.00        59

    accuracy                           1.00    250998
   macro avg       0.50      0.50      0.50    250998
weighted avg       1.00      1.00      1.00    250998



/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [6]:
df = pd.read_csv("../../labelled_data/part_D/partd_rus_features.csv")

In [7]:
df = df.drop(["Unnamed: 0","Unnamed: 0.1","PRSCRBR_NPI"],axis=1)

In [8]:
df

,Prscrbr_Crdntls,Prscrbr_Gndr,Prscrbr_Type,Prscrbr_Type_src,Tot_Clms,Tot_30day_Fills,Tot_Drug_Cst,Tot_Day_Suply,Tot_Benes,GE65_Sprsn_Flag,...,Bene_Race_Wht_Cnt,Bene_Race_Black_Cnt,Bene_Race_Api_Cnt,Bene_Race_Hspnc_Cnt,Bene_Race_Natind_Cnt,Bene_Race_Othr_Cnt,Bene_Dual_Cnt,Bene_Ndual_Cnt,Bene_Avg_Risk_Scre,Fraud
0,2929,1.0,120,1.0,1132.0,1191.566667,71404.48,25695.0,103.0,1.0,...,97.0,0.0,0.0,0.0,30.0,0.0,37.0,66.0,1.488790,0
1,4394,0.0,131,1.0,201.0,203.400000,2945.42,1641.0,77.0,1.0,...,69.0,20.0,0.0,0.0,0.0,0.0,13.0,80.0,1.040951,0
2,5885,0.0,86,1.0,7058.0,14652.966667,841439.84,428631.0,413.0,1.0,...,338.0,38.0,0.0,28.0,0.0,11.0,116.0,297.0,1.375367,0
3,9176,0.0,64,1.0,186.0,244.000000,4984.06,4018.0,105.0,1.0,...,80.0,17.0,0.0,0.0,0.0,0.0,32.0,73.0,1.393905,0
4,1060,1.0,120,1.0,54.0,96.000000,2551.76,2813.0,17.0,1.0,...,13.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,1.222294,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,7632,0.0,63,1.0,263.0,423.600000,66723.17,12370.0,55.0,1.0,...,39.0,38.0,35.0,14.0,0.0,0.0,16.0,20.0,2.249708,1
580,5885,0.0,60,1.0,813.0,1618.800000,56694.66,45859.0,85.0,1.0,...,100.0,23.0,27.0,23.0,0.0,14.0,38.0,47.0,0.956362,1
581,7632,0.0,63,0.0,1040.0,2311.366667,100240.86,67529.0,122.0,1.0,...,44.0,30.0,0.0,105.0,0.0,0.0,101.0,21.0,1.233241,1
582,9176,0.0,86,1.0,2132.0,2296.933333,1444745.29,64849.0,90.0,1.0,...,59.0,11.0,15.0,15.0,0.0,14.0,49.0,41.0,1.292911,1


In [9]:
X = df.drop(['Fraud'], axis=1)
y = df['Fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Convert the data to DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set hyperparameters for the XGBoost model
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 3,
    'eta': 0.1,
    'subsample': 0.8,
}

# Train the XGBoost model
num_round = 100
model = xgb.train(params, dtrain, num_round)

# Make predictions on the test set
y_pred = model.predict(dtest)

# Convert predicted probabilities to binary predictions
y_pred = [1 if pred > 0.5 else 0 for pred in y_pred]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", classification_rep)

Accuracy: 0.7094017094017094

Confusion Matrix:
 [[41 18]
 [16 42]]

Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.69      0.71        59
           1       0.70      0.72      0.71        58

    accuracy                           0.71       117
   macro avg       0.71      0.71      0.71       117
weighted avg       0.71      0.71      0.71       117



In [10]:
df = pd.read_csv("../../labelled_data/part_D/partd_ros_features.csv")

In [11]:
df = df.drop(["Unnamed: 0","Unnamed: 0.1","PRSCRBR_NPI"],axis=1)

In [12]:
df

,Prscrbr_Crdntls,Prscrbr_Gndr,Prscrbr_Type,Prscrbr_Type_src,Tot_Clms,Tot_30day_Fills,Tot_Drug_Cst,Tot_Day_Suply,Tot_Benes,GE65_Sprsn_Flag,...,Bene_Race_Wht_Cnt,Bene_Race_Black_Cnt,Bene_Race_Api_Cnt,Bene_Race_Hspnc_Cnt,Bene_Race_Natind_Cnt,Bene_Race_Othr_Cnt,Bene_Dual_Cnt,Bene_Ndual_Cnt,Bene_Avg_Risk_Scre,Fraud
0,7632,0.0,86,1.0,324.0,369.800000,20606.08,8621.0,106.0,1.0,...,67.0,27.0,0.0,13.0,0.0,0.0,28.0,78.0,2.245800,0
1,7632,0.0,9,1.0,1992.0,2145.666667,79803.65,60953.0,228.0,1.0,...,130.0,81.0,0.0,12.0,0.0,0.0,125.0,103.0,1.695165,0
2,4394,0.0,42,1.0,57.0,57.700000,327.34,554.0,43.0,1.0,...,41.0,0.0,0.0,0.0,0.0,0.0,0.0,43.0,1.006070,0
3,3535,1.0,42,1.0,18.0,18.000000,128.52,181.0,16.0,1.0,...,12.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,1.251869,0
4,6421,1.0,120,1.0,37.0,47.000000,3834.98,1366.0,17.0,1.0,...,12.0,0.0,0.0,0.0,0.0,0.0,11.0,11.0,4.919074,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2509385,9176,0.0,86,1.0,6364.0,11801.166667,478117.79,342993.0,312.0,1.0,...,236.0,41.0,0.0,22.0,0.0,13.0,60.0,252.0,1.169697,1
2509386,9176,0.0,60,1.0,5599.0,10111.033333,538281.10,296379.0,444.0,1.0,...,393.0,66.0,13.0,32.0,0.0,17.0,233.0,211.0,1.369505,1
2509387,9176,0.0,60,1.0,106.0,109.333333,3824.90,3195.0,33.0,1.0,...,33.0,0.0,0.0,0.0,0.0,0.0,12.0,21.0,2.111492,1
2509388,7632,0.0,86,1.0,959.0,1501.533333,107461.83,40631.0,134.0,1.0,...,53.0,18.0,14.0,42.0,0.0,19.0,80.0,54.0,1.624990,1


In [13]:
X = df.drop(['Fraud'], axis=1)
y = df['Fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Convert the data to DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set hyperparameters for the XGBoost model
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 3,
    'eta': 0.1,
    'subsample': 0.8,
}

# Train the XGBoost model
num_round = 100
model = xgb.train(params, dtrain, num_round)

# Make predictions on the test set
y_pred = model.predict(dtest)

# Convert predicted probabilities to binary predictions
y_pred = [1 if pred > 0.5 else 0 for pred in y_pred]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", classification_rep)

Accuracy: 0.897030752493634

Confusion Matrix:
 [[212694  38067]
 [ 13611 237506]]

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.85      0.89    250761
           1       0.86      0.95      0.90    251117

    accuracy                           0.90    501878
   macro avg       0.90      0.90      0.90    501878
weighted avg       0.90      0.90      0.90    501878



In [14]:
df = pd.read_csv("../../labelled_data/part_D/partd_smote_features.csv")

In [15]:
df = df.drop(["Unnamed: 0","Unnamed: 0.1","PRSCRBR_NPI"],axis=1)

In [16]:
X = df.drop(['Fraud'], axis=1)
y = df['Fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Convert the data to DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set hyperparameters for the XGBoost model
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 3,
    'eta': 0.1,
    'subsample': 0.8,
}

# Train the XGBoost model
num_round = 100
model = xgb.train(params, dtrain, num_round)

# Make predictions on the test set
y_pred = model.predict(dtest)

# Convert predicted probabilities to binary predictions
y_pred = [1 if pred > 0.5 else 0 for pred in y_pred]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", classification_rep)

Accuracy: 0.9909449152371465

Confusion Matrix:
 [[245707     85]
 [  4405 245657]]

Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99    245792
           1       1.00      0.98      0.99    250062

    accuracy                           0.99    495854
   macro avg       0.99      0.99      0.99    495854
weighted avg       0.99      0.99      0.99    495854



In [17]:
df = pd.read_csv("../../labelled_data/part_D/partd_rus_type_features.csv")

In [18]:
df = df.drop(["Unnamed: 0","Unnamed: 0.1","PRSCRBR_NPI"],axis=1)

In [20]:

# Assuming df is your DataFrame containing the data

X = df.drop(['FraudType'], axis=1)
y = df['FraudType']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data to DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set hyperparameters for the XGBoost model
params = {
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',  # Use 'mlogloss' for multi-class problems
    'max_depth': 3,
    'eta': 0.1,
    'subsample': 0.8,
    'num_class': len(pd.unique(y)),  # Specify the number of classes
}

# Train the XGBoost model
num_round = 100
model = xgb.train(params, dtrain, num_round)

# Make predictions on the test set
y_pred_prob = model.predict(dtest)
y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", classification_rep)


Accuracy: 0.0

Confusion Matrix:
 [[0 1 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 1 0 0]]

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           5       0.00      0.00      0.00       0.0
           9       0.00      0.00      0.00       1.0
          11       0.00      0.00      0.00       1.0

    accuracy                           0.00       3.0
   macro avg       0.00      0.00      0.00       3.0
weighted avg       0.00      0.00      0.00       3.0



/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modi

In [2]:
df = pd.read_csv("../../labelled_data/part_D/partd_ros_type_features.csv")

In [3]:
df = df.drop(["Unnamed: 0","Unnamed: 0.1","PRSCRBR_NPI"],axis=1)

In [4]:
df

,Prscrbr_Crdntls,Prscrbr_Gndr,Prscrbr_Type,Prscrbr_Type_src,Tot_Clms,Tot_30day_Fills,Tot_Drug_Cst,Tot_Day_Suply,Tot_Benes,GE65_Sprsn_Flag,...,Bene_Race_Wht_Cnt,Bene_Race_Black_Cnt,Bene_Race_Api_Cnt,Bene_Race_Hspnc_Cnt,Bene_Race_Natind_Cnt,Bene_Race_Othr_Cnt,Bene_Dual_Cnt,Bene_Ndual_Cnt,Bene_Avg_Risk_Scre,FraudType
0,7632,0.0,86,1.0,324.0,369.800000,20606.08,8621.0,106.0,1.0,...,67.0,27.0,0.0,13.0,0.0,0.0,28.0,78.0,2.245800,0
1,7632,0.0,9,1.0,1992.0,2145.666667,79803.65,60953.0,228.0,1.0,...,130.0,81.0,0.0,12.0,0.0,0.0,125.0,103.0,1.695165,0
2,4394,0.0,42,1.0,57.0,57.700000,327.34,554.0,43.0,1.0,...,41.0,0.0,0.0,0.0,0.0,0.0,0.0,43.0,1.006070,0
3,3535,1.0,42,1.0,18.0,18.000000,128.52,181.0,16.0,1.0,...,12.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,1.251869,0
4,6421,1.0,120,1.0,37.0,47.000000,3834.98,1366.0,17.0,1.0,...,12.0,0.0,0.0,0.0,0.0,0.0,11.0,11.0,4.919074,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17565725,6109,1.0,158,1.0,167.0,206.000000,14205.65,4626.0,78.0,1.0,...,54.0,22.0,0.0,0.0,0.0,0.0,13.0,65.0,1.319562,13
17565726,6109,1.0,158,1.0,167.0,206.000000,14205.65,4626.0,78.0,1.0,...,54.0,22.0,0.0,0.0,0.0,0.0,13.0,65.0,1.319562,13
17565727,6109,1.0,158,1.0,167.0,206.000000,14205.65,4626.0,78.0,1.0,...,54.0,22.0,0.0,0.0,0.0,0.0,13.0,65.0,1.319562,13
17565728,6109,1.0,158,1.0,167.0,206.000000,14205.65,4626.0,78.0,1.0,...,54.0,22.0,0.0,0.0,0.0,0.0,13.0,65.0,1.319562,13


In [5]:

# Assuming df is your DataFrame containing the data

X = df.drop(['FraudType'], axis=1)
y = df['FraudType']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data to DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set hyperparameters for the XGBoost model
params = {
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',  # Use 'mlogloss' for multi-class problems
    'max_depth': 3,
    'eta': 0.1,
    'subsample': 0.8,
    'num_class': len(pd.unique(y)),  # Specify the number of classes
}

# Train the XGBoost model
num_round = 100
model = xgb.train(params, dtrain, num_round)

# Make predictions on the test set
y_pred_prob = model.predict(dtest)
y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", classification_rep)


: 